In [1]:
# Import libraries
import os
import easygui

# Import helper functions
from detect_audio_triggers import *
from sync_eeg_data import *
from sync_simulator_data import *


In [2]:
# Call the function to detect audio triggers
trigger_time = detect_audio_triggers(segment_length = 8, hz_target = 9000)

Selected peak time: 5.57 seconds


In [3]:
# Correct for delay peaks if necessary
delay = 0
trigger_time = trigger_time - (0.5 * delay)

In [6]:
# Sync the EEG data
eeg_sync_data = sync_eeg_data(trigger_time = trigger_time)
eeg_sync_data

c:\Users\rreareyes\Documents\driving_simulator\bin\sync_eeg_data.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_sync_data["sync_time_stamp"] = eeg_sync_data["time_stamp"] - eeg_sync_time_stamp


,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,...,FC5,FC6,CP5,CP6,FT9,FT10,TP9,TP10,time_stamp,sync_time_stamp
1955,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,586.738713,0.000000
1956,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,586.742713,0.004000
1957,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,586.746713,0.008000
1958,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,586.750713,0.012000
1959,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,586.754713,0.016000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19022,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,655.006757,68.268044
19023,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,655.010757,68.272044
19024,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,655.014757,68.276044
19025,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,...,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,187499.984375,655.018757,68.280044


In [ ]:
# Sync the Simulator data
sim_sync_data = sync_simulator_data(trigger_time = trigger_time)
sim_sync_data

In [ ]:
# Export synced data
dir_synchronized = easygui.diropenbox(title = "Select folder to export synchronized data")

# Ask for experiment name
experiment_basename = easygui.enterbox(msg = "Enter experiment name", title = "Experiment name")


eeg_sync_data.to_csv(os.path.join(dir_synchronized, experiment_basename + "_eeg_sync.csv"), index=False)
sim_sync_data.to_csv(os.path.join(dir_synchronized, experiment_basename + "_sim_sync.csv"), index=False)